<a href="https://colab.research.google.com/github/choi-yh/DataMining/blob/master/8_1_rnn_hello_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Hidden layer 확인

In [ ]:
import numpy as np
from tensorflow.keras import layers
# One hot encoding for each chare in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# One cell RNN input_dim (4) -> output_dim (3)
x_data = np.array([[h]], dtype=np.float32)

hidden_size = 3
rnn = layers.SimpleRNN(units=hidden_size, return_sequences=True, return_state=True)
outputs, states = rnn(x_data)

print(x_data, x_data.shape)
print(outputs, outputs.shape)

[[[1. 0. 0. 0.]]] (1, 1, 4)
tf.Tensor([[[-0.24390677  0.25269923  0.61246765]]], shape=(1, 1, 3), dtype=float32) (1, 1, 3)


In [ ]:
# One cell RNN input_dim (4) -> output_dim (3). sequence: 5
x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
print(x_data, x_data.shape)

hidden_size = 3
rnn = layers.SimpleRNN(units=hidden_size, return_sequences=True, return_state=True)
outputs, states = rnn(x_data)

print(outputs, outputs.shape)
print(states, states.shape)

[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]] (1, 5, 4)
tf.Tensor(
[[[ 0.71114606 -0.18268691 -0.28990424]
  [-0.15831852  0.74646443 -0.89320624]
  [ 0.22950675 -0.8219949  -0.75164616]
  [ 0.27212977  0.37423676 -0.47952795]
  [-0.06988575  0.2509352  -0.53459734]]], shape=(1, 5, 3), dtype=float32) (1, 5, 3)
tf.Tensor([[-0.06988575  0.2509352  -0.53459734]], shape=(1, 3), dtype=float32) (1, 3)


In [ ]:
x_data = x_data[:5]
y_data = x_data[1:]

print(x_data, y_data)

[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]] []


#### many to many

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [ ]:
text = 'hello'

In [ ]:
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)
print(char_vocab)
print('글자 집합의 크기: {}'.format(vocab_size))

['e', 'h', 'l', 'o']
글자 집합의 크기: 4


In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{'e': 0, 'h': 1, 'l': 2, 'o': 3}


In [ ]:
_tmp = []
for ch in text:
    _tmp.append(char_to_index[ch])
_tmp

[1, 0, 2, 2, 3]

In [ ]:
train_X = [_tmp[:4]]
train_y = [_tmp[1:]]
train_X, train_y

([[1, 0, 2, 2]], [[0, 2, 2, 3]])

In [ ]:
seq_length = 4
n_samples = 1

In [ ]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)
print(train_X.shape)
print(train_X)

(1, 4, 3)
[[[0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]]]


In [ ]:
n_hidden = 5
model = Sequential()
# return_sequences=True 옵션을 통해 cost를 many2many 방식으로 계산하도록 한다.
model.add(layers.SimpleRNN(n_hidden, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax')))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=200, verbose=2)

Epoch 1/200
1/1 [==============================] - 0s 1ms/step - loss: 0.8791 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8759 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8727 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 0.8694 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 0.8662 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 1ms/step - loss: 0.8630 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8598 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8566 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8534 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 1ms/step - loss: 0.8503 - accuracy: 1.0000
Epoch 11/200
1/1 [===========

In [ ]:
# 학습된 모형을 이용해 'hell'을 입력으로 예측한 결과, [0,2,2,3] 즉 'ello'가 출력되었다.
pred = model.predict(train_X)
pred

array([[[0.35680637, 0.1151687 , 0.34191668, 0.18610834],
        [0.15888003, 0.10878655, 0.5090944 , 0.22323911],
        [0.3060231 , 0.11748935, 0.33800042, 0.2384871 ],
        [0.14248234, 0.13979639, 0.23394766, 0.4837736 ]]], dtype=float32)

In [ ]:
np.argmax(pred, 2)

array([[0, 2, 2, 3]])